In [ ]:
#write out the algorithm, check that it works
from random import uniform
from math import sqrt
from time import time

#Define total number of darsts to throw - perhaps make this an input value
num_darts = 2000000
#Define a variable to store the number of darts that fall inside the circle
num_darts_circle = 0

#Record the time execution time of the loop that runs the actual situation
start_time = time()

#This loop simulates the dart throwing. For ea. dart find a random position in the unit square for it to fall
#Test if the location of the thrown dart is within the circle by calculating its distance from the center (0.5,0.5)
#Darts that are within 0.5 of the origin are within the circle.
for n in range(num_darts):
    x,y = uniform(0,1), uniform(0,1)
    if sqrt((x-0.5)**2+(y-0.5)**2) <= 0.5:
        num_darts_circle += 1

#record the time after the conclusion of the loop
end_time = time()
#The total time requiered to run the loop is the difference
execution_time = end_time - start_time

#Calc. the approx for Pi
# pi = 4 * Area
pi_approx = 4 * (num_darts_circle / float(num_darts))

print ("Pi Approximation:", pi_approx)
print ("Number of Darts:", num_darts)
print ("Execution Time (s):", execution_time)
print ("Darts Thrown per Second:", num_darts/execution_time)

In [ ]:
#3 diff parallization methods
# 1 - threading
# 2 - multiprocessor/pooling
# 3 - concurrent futures

In [1]:
#Threading Method

from random import uniform
from math import sqrt
from time import time
import os
os.getpid()
import threading
import numpy as np
from queue import Queue

#initialize export variables
a = Queue()
b = Queue()
c = Queue()

#define the simulation in a threading method
def thrower(num):
    """Dart Throwing code"""
    #Define total number of darts to throw
    num_darts = num
    #Define a variable to store the number of darts that fall inside the circle
    num_darts_circle = 0

    #Record the time execution time of the loop that runs the actual situation
    start_time = time()

    #This loop simulates the dart throwing. For ea. dart find a random position in the unit square for it to fall
    #Test if the location of the thrown dart is within the circle by calculating its distance from the center (0.5,0.5)
    #Darts that are within 0.5 of the origin are within the circle.
    for n in range(num_darts):
        x,y = uniform(0,1), uniform(0,1)
        if sqrt((x-0.5)**2+(y-0.5)**2) <= 0.5:
            num_darts_circle += 1

    #record the time after the conclusion of the loop
    end_time = time()
    #The total time requiered to run the loop is the difference
    execution_time = end_time - start_time

    #Calc. the approx for Pi
    # pi = 4 * Area
    pi_approx = 4 * (num_darts_circle / float(num_darts))
    darts_per_sec = num_darts/execution_time
    
    #output results
    a.put(darts_per_sec)
    b.put(execution_time)
    c.put(num_darts)
    return

threads = []
#creat the input values for the threads
trials = np.linspace(10**1,10**8,71)
vals=[]
for i in range(len(trials)):
    vals.append(int(trials[i]))

#call the threading function
for x,i in enumerate(vals):
    t = threading.Thread(target=thrower, args=(i,))
    threads.append(t)
    t.start()
    

sim_rate = list(a.queue)
Execution_time = list(b.queue)
Darts_thrown = list(c.queue)

In [2]:
# #multiprocessing - Don't run this one
# #Splits threads across processes

# import multiprocessing
# from random import uniform
# from math import sqrt
# from time import time
# import os
# os.getpid()
# import numpy as np

# #define the simulation in a multiprocessing method
# def thrower(num):
#     """Dart Throwing code"""
#     #Define total number of darts to throw
#     num_darts = num
#     #Define a variable to store the number of darts that fall inside the circle
#     num_darts_circle = 0

#     #Record the time execution time of the loop that runs the actual situation
#     start_time = time()

#     #This loop simulates the dart throwing. For ea. dart find a random position in the unit square for it to fall
#     #Test if the location of the thrown dart is within the circle by calculating its distance from the center (0.5,0.5)
#     #Darts that are within 0.5 of the origin are within the circle.
#     for n in range(num_darts):
#         x,y = uniform(0,1), uniform(0,1)
#         if sqrt((x-0.5)**2+(y-0.5)**2) <= 0.5:
#             num_darts_circle += 1

#     #record the time after the conclusion of the loop
#     end_time = time()
#     #The total time requiered to run the loop is the difference
#     execution_time = end_time - start_time

#     #Calc. the approx for Pi
#     # pi = 4 * Area
#     pi_approx = 4 * (num_darts_circle / float(num_darts))
#     darts_per_sec = num_darts/execution_time
    
#     return (num_darts, execution_time, darts_per_sec)

# procs = []
# #create the input values for the threads
# trials = np.linspace(10**1,10**7,71)
# vals=[]
# for i in range(len(trials)):
#     vals.append(int(trials[i]))

# #call the multiprocessor
# for i in vals:
#     p = multiprocessing.Process(target=thrower, args=(i,))
#     procs.append(p)
#     p.start()


In [3]:
#Pooling w/multiprocessing
#Start separate pools
import time
import multiprocessing
from multiprocessing import Pool
from random import uniform
from math import sqrt
from time import time
import os
os.getpid()
import numpy as np

def thrower(num):
    """Dart Throwing code"""
    #Define total number of darts to throw
    num_darts = num
    #Define a variable to store the number of darts that fall inside the circle
    num_darts_circle = 0
    #Record the time execution time of the loop that runs the actual situation
    start_time = time()

    #This loop simulates the dart throwing. For ea. dart find a random position in the unit square for it to fall
    #Test if the location of the thrown dart is within the circle by calculating its distance from the center (0.5,0.5)
    #Darts that are within 0.5 of the origin are within the circle.
    for n in range(num_darts):
        x,y = uniform(0,1), uniform(0,1)
        if sqrt((x-0.5)**2+(y-0.5)**2) <= 0.5:
            num_darts_circle += 1

    #record the time after the conclusion of the loop
    end_time = time()
    #The total time requiered to run the loop is the difference
    execution_time = end_time - start_time

    #Calc. the approx for Pi
    # pi = 4 * Area
    pi_approx = 4 * (num_darts_circle / float(num_darts))
    darts_per_sec = num_darts/execution_time
    
    return (num_darts, execution_time, darts_per_sec)

trials = np.linspace(10**1,10**7,71)
vals=[]
for i in range(len(trials)):
    vals.append(int(trials[i]))


pool = Pool(processes=8)  # start worker processes 
ress = pool.map(thrower,vals) #call the pool to execute the simulation function
pool.terminate() 
del pool

#parse results (list of tuples) likely not the most elegant way to do this...
pl_Darts_thrown=[]
pl_sim_rate=[]
pl_execution_time=[]
for i in range(len(ress)):
        pl_Darts_thrown.append(ress[i][0])
        pl_sim_rate.append(ress[i][2])
        pl_execution_time.append(ress[i][1])

In [4]:
#concurrent.futures
from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor()

def thrower(num):
    """Dart Throwing code"""
    #Define total number of darts to throw
    num_darts = num
    #Define a variable to store the number of darts that fall inside the circle
    num_darts_circle = 0

    #Record the time execution time of the loop that runs the actual situation
    start_time = time()

    #This loop simulates the dart throwing. For ea. dart find a random position in the unit square for it to fall
    #Test if the location of the thrown dart is within the circle by calculating its distance from the center (0.5,0.5)
    #Darts that are within 0.5 of the origin are within the circle.
    for n in range(num_darts):
        x,y = uniform(0,1), uniform(0,1)
        if sqrt((x-0.5)**2+(y-0.5)**2) <= 0.5:
            num_darts_circle += 1

    #record the time after the conclusion of the loop
    end_time = time()
    #The total time requiered to run the loop is the difference
    execution_time = end_time - start_time

    #Calc. the approx for Pi
    # pi = 4 * Area
    pi_approx = 4 * (num_darts_circle / float(num_darts))
    darts_per_sec = num_darts/execution_time
    
    return (num_darts, execution_time, darts_per_sec)


trials = np.linspace(10**1,10**7,71)
vals=[]
for i in range(len(trials)):
    vals.append(int(trials[i]))

future = e.submit(thrower,vals)
results = list(e.map(thrower, vals))

#parse results (list of tuples) likely not the most elegant way to do this...
f_Darts_thrown=[]
f_sim_rate=[]
f_execution_time=[]
for i in range(len(results)):
        f_Darts_thrown.append(results[i][0])
        f_sim_rate.append(results[i][2])
        f_execution_time.append(results[i][1])

e.shutdown()



In [ ]:
#plot results of the different parallelization methods
%matplotlib inline
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize = (12,10))

pt1 = ax.plot(Darts_thrown,Execution_time,'b',label = 'Threading')
pt2 = ax.plot(pl_Darts_thrown,pl_execution_time,'r',label = 'Multiprocessing - Pooling')
pt3 = ax.plot(f_Darts_thrown,f_execution_time,'g',label = 'Concurrent Futures')
ax.set_ylabel('Execution Time (sec)',fontsize = 16)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Darts Thrown',fontsize = 16)

ax2 = ax.twinx()
pt4 = ax2.plot(Darts_thrown,sim_rate,'b-.',label = 'Multiprocessing')
pt5 = ax2.plot(pl_Darts_thrown,pl_sim_rate,'r-.',label = 'Multiprocessing - Pooling')
pt6 = ax.plot(f_Darts_thrown,f_sim_rate,'g-.',label = 'Concurrent Futures')
ax2.set_yscale('log')
ax2.set_ylabel('Simulation Rate (darts/sec) dashed line',fontsize = 16)

pt = pt1+pt2+pt3
pts = [l.get_label() for l in pt]
ax.legend(pt, pts, loc='center left',frameon = False)


